<a href="https://colab.research.google.com/github/Abdulaziz1607/Challenge_Data/blob/main/Dreem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dreem

---

# Detecting Sleep Apnea from raw physiological signals


# Bibli

In [ ]:
!pip install tslearn

In [ ]:
import pandas as pd, seaborn as sns, numpy as np, matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")
sns.set()
from tslearn.clustering import TimeSeriesKMeans
import joblib
from sklearn.covariance import oas
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import gzip

In [ ]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM,Input, Dense, Conv1D, Concatenate, Dropout, Subtract, \
                        Flatten, MaxPooling1D, Multiply, Lambda, Add, Dot,Minimum,Reshape,AveragePooling1D,TimeDistributed,MaxPooling2D, Conv2D
from keras.backend import constant
from keras import optimizers
import keras
from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import Input
from keras import initializers
from keras.utils import to_categorical
from keras.constraints import max_norm
import keras.backend as K
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from keras import regularizers

import lightgbm as lgb
import h5py

In [ ]:
cd drive/My Drive/Colab Notebooks/Dreem

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/Dreem'
/content/drive/My Drive/Colab Notebooks/Dreem


# Importer les données

In [ ]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

sub = pd.read_csv("y_benchmark.csv")

In [ ]:
X_train_w = h5py.File('X_train.h5', 'r')
X_test_w = h5py.File('X_test.h5', 'r')

In [ ]:
X_train_w =pd.DataFrame(X_train_w["data"][:])
X_test_w = pd.DataFrame(X_test_w["data"][:])

In [ ]:
X_train_w.drop([0,1],axis=1,inplace=True)
X_test_w.drop([0,1],axis=1,inplace=True)

In [ ]:
y_train.drop(["ID"],axis=1,inplace=True)

# Metric

In [ ]:
def jaccard_overlap(localizations_a, localizations_b):
    """Jaccard overlap between two segments A ∩ B / (LENGTH_A + LENGTH_B - A ∩ B)

    localizations_a: list of localizations
    localizations_a: list of localizations

    return:
        (ndarray)
    """
    size_set_A = len(localizations_a)
    size_set_B = len(localizations_b)

    localizations_a_start = [localization[0] for localization in localizations_a]
    localizations_a_end = [localization[1] for localization in localizations_a]

    localizations_b_start = [localization[0] for localization in localizations_b]
    localizations_b_end = [localization[1] for localization in localizations_b]

    localizations_a_start = np.array([localizations_a_start for _ in range(size_set_B)])
    localizations_a_end = np.array([localizations_a_end for _ in range(size_set_B)])

    localizations_b_start = np.transpose(np.array([localizations_b_start for _ in range(size_set_A)]))
    localizations_b_end = np.transpose(np.array([localizations_b_end for _ in range(size_set_A)]))

    length_a = localizations_a_end - localizations_a_start
    length_b = localizations_b_end - localizations_b_start


    # intersection
    max_min = np.maximum(localizations_a_start, localizations_b_start)
    min_max = np.minimum(localizations_a_end, localizations_b_end)
    intersection = np.maximum((min_max - max_min), 0)

    try:
        overlaps = intersection / (length_a + length_b - intersection)
    except:
        print('here')
    return overlaps


def extract_events_from_binary_mask(binary_mask, fs=1):
    binary_mask = np.array([0] + binary_mask.tolist() + [0])
    diff_data = np.diff(binary_mask)
    starts = np.where(diff_data == 1)[0] / fs
    ends = np.where(diff_data == -1)[0] / fs

    assert len(starts) == len(ends)
    events = []
    for i, _ in enumerate(starts):
        events += [(starts[i], ends[i])]

    return events


def compute_f1_score(event_pred, event_true):
    total_tp = 0
    total_fp = 0
    total_fn = 0
    for idx in range(event_pred.shape[0]):
        tp, fp, fn = compute_tp_fp_fn_for_each_entry(event_pred[idx], event_true[idx])
        total_tp += tp
        total_fp += fp
        total_fn +=fn

    precision = total_tp / (total_tp + total_fp)
    recall = total_tp / (total_tp + total_fn)
    if precision == 0 or recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * precision * recall / (precision + recall)

    return f1_score



def compute_tp_fp_fn_for_each_entry(prediction, reference, min_iou=0.3):
    """takes 2 event scorings
        (in array format [[start1, end1], [start2, end2], ...])
        and outputs the f1 score.

        Parameters
        ----------
        min_iou : float
            minimum intersection-over-union with a true event to be considered
            a true positive.
        """
    if len(prediction) == 0:
        return 0, 0, len(reference)
    if len(reference) == 0:
       return 0, len(prediction), 0

    iou = jaccard_overlap(prediction, reference)

    # Number of true events which are matched by a predicted events
    TP1 = np.sum(np.amax((iou >= min_iou), axis=0))
    # Number of predicted events which match a true event
    TP2 = np.sum(np.amax((iou >= min_iou), axis=1))

    # In order to avoid duplicate, a true event can only match a true events and a predicted event
    true_positive = min(TP1, TP2)
    false_positive = len(prediction) - true_positive
    false_negative = len(reference) - true_positive

    return true_positive, false_positive, false_negative



def format_predictions_for_scoring(mask, window_length=100):
    """
    Stack events in one single list for F1 computation
    :param mask:
    :param window_length:
    :return:
    """
    result = []
    for i, elt in enumerate(mask):
        events = extract_events_from_binary_mask(elt)
        if len(events) > 0:
            events = [[i * window_length + start, i * window_length + end] for start, end in events]
        else:
            events = []
        result.append(events)
    return np.array(result)


def dreem_sleep_apnea_custom_metric(y_pred, y_true):
    event_true = format_predictions_for_scoring(np.array(y_pred))
    event_pred = format_predictions_for_scoring(np.array(y_true))
    return compute_f1_score(event_pred, event_true)

# Visu Fonctions

In [ ]:
SIGNALS_NAME = [
    "AbdoBelt",
    "AirFlow",
    "PPG",
    "ThorBelt",
    "Snoring",
    "SPO2",
    "C4A1",
    "O2A1",
]


def extract_events_from_binary_mask(binary_mask, fs=1):
    binary_mask = np.array([0] + binary_mask.tolist() + [0])
    diff_data = np.diff(binary_mask)
    starts = np.where(diff_data == 1)[0] / fs
    ends = np.where(diff_data == -1)[0] / fs

    assert len(starts) == len(ends)
    events = []
    for i, _ in enumerate(starts):
        events += [(starts[i], ends[i])]

    return events


def visualize_signal_and_event(X, mask, signals_name=SIGNALS_NAME, signal_freq=100):
    n_signal = X.shape[0]
    fig, axs = plt.subplots(n_signal, sharex=True)
    events = extract_events_from_binary_mask(mask)
    for i in range(n_signal):
        axs[i].plot(np.arange(0, X[i].shape[0]) / signal_freq, X[i])
        axs[i].set_ylabel(signals_name[i])
        for elt in events:
            axs[i].axvspan(elt[0], elt[1], color='red', alpha=0.3)
    plt.xlim(0, X[0].shape[0]/ signal_freq)
    plt.show()


def visualise_index(idx, data_h5, masks, N_signals=8):
    x = data_h5['data'][idx, 2:]
    x = x.reshape(N_signals, -1)
    visualize_signal_and_event(x, np.array(masks[idx, 1:]))
"""
if __name__ == "__main__":
    import h5py
    PATH_TO_TRAINING_DATA = ""
    PATH_TO_TRAINING_TARGET = ""
    h5_file = h5py.File(PATH_TO_TRAINING_DATA)
    mask = np.array(pd.read_csv(PATH_TO_TRAINING_TARGET))
    visualise_index(0, h5_file, mask)
"""

'\nif __name__ == "__main__":\n    import h5py\n    PATH_TO_TRAINING_DATA = ""\n    PATH_TO_TRAINING_TARGET = ""\n    h5_file = h5py.File(PATH_TO_TRAINING_DATA)\n    mask = np.array(pd.read_csv(PATH_TO_TRAINING_TARGET))\n    visualise_index(0, h5_file, mask)\n'

# Analyse

In [ ]:
list_ind = []
for i in range(y_train.shape[0]):
  if y_train.iloc[i].sum() != 0:
    list_ind.append(i)

In [ ]:
y_train_apne = y_train.iloc[list_ind]
X_train_apne = X_train_w.iloc[list_ind]

X_train_w = np.array(X_train_w)
X_test_w = np.array(X_test_w)

In [ ]:
sc = StandardScaler()
for i in range(8):
  
  X_train_w[:,9*1000*i:9*1000*(i+1)] = sc.fit_transform(X_train_w[:,9*1000*i:9*1000*(i+1)])
  X_test_w[:,9*1000*i:9*1000*(i+1)] = sc.transform(X_test_w[:,9*1000*i:9*1000*(i+1)])

In [ ]:
X_train_w = X_train_w.reshape(X_train_w.shape[0],8,9000,1)
X_test_w = X_test_w.reshape(X_test_w.shape[0],8,9000,1)

X_train_apne = np.array(X_train_apne).reshape(X_train_apne.shape[0],8,9000,1)

# Modèle

In [ ]:
y_train_ = to_categorical(y_train_apne)

In [ ]:
X,X_vali,y,y_vali = train_test_split(X_train_apne,y_train_,test_size=0.3,random_state=123)

In [ ]:
y.shape

(630, 90, 2)

In [ ]:
X.shape

(630, 8, 9000, 1)

In [ ]:
n_timesteps, n_features, n_outputs = X.shape[2], X.shape[1], y.shape[1]

In [ ]:
model = Sequential()
model.add(Input(shape=(n_features,n_timesteps,1)))
model.add(MaxPooling2D(pool_size=(1,10)))
model.add(MaxPooling2D(pool_size=(1,10)))
model.add(Conv2D(filters=2, kernel_size=(8,1), activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_7 (MaxPooling2 (None, 8, 900, 1)         0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 90, 1)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 90, 2)          18        
Total params: 18
Trainable params: 18
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y = np.array(y).reshape(y.shape[0], 1,90,2)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
model.fit(X, y, epochs=20, verbose=True)

Epoch 1/20
20/20 [==============================] - 2s 48ms/step - loss: 5824.1940 - mean_io_u_1: 0.4767
Epoch 2/20
20/20 [==============================] - 1s 47ms/step - loss: 5398.0479 - mean_io_u_1: 0.3667
Epoch 3/20
20/20 [==============================] - 1s 47ms/step - loss: 5179.5333 - mean_io_u_1: 0.3912
Epoch 4/20
20/20 [==============================] - 1s 48ms/step - loss: 4875.8590 - mean_io_u_1: 0.3989
Epoch 5/20
20/20 [==============================] - 1s 49ms/step - loss: 4432.8809 - mean_io_u_1: 0.4167
Epoch 6/20
20/20 [==============================] - 1s 48ms/step - loss: 4141.4597 - mean_io_u_1: 0.4070
Epoch 7/20
20/20 [==============================] - 1s 47ms/step - loss: 3842.7178 - mean_io_u_1: 0.4061
Epoch 8/20
20/20 [==============================] - 1s 47ms/step - loss: 3648.6821 - mean_io_u_1: 0.3958
Epoch 9/20
20/20 [==============================] - 1s 47ms/step - loss: 3217.6494 - mean_io_u_1: 0.4073
Epoch 10/20
20/20 [==============================] - 1s

In [ ]:
model.save('my_model.h5')  

In [ ]:
#model = keras.load_model('my_model.h5')

# Prédiction

In [ ]:
pred = model.predict(X_vali)
pred = pred.reshape(pred.shape[0],90,2)
pred = np.argmax(pred,axis=2)

In [ ]:
pd.DataFrame(pred).sum().sum()

5455

In [ ]:
y_vali_ = y_vali.reshape(pred.shape[0],90,2)
y_vali_ =np.argmax(y_vali_,axis=2)

In [ ]:
pd.DataFrame(y_vali_).sum().sum()

8294

In [ ]:
pd.DataFrame(pred)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0,1,1,0,0,0,1,1,1,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,0,1,1,1,0,0,1,0,0,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,1,0,0,1,1,1,1,0,0,1,1,0,0,1,1,1,...,1,1,1,0,1,1,1,0,0,1,1,0,1,0,0,1,1,0,1,1,1,0,0,1,1,1,0,1,1,1,0,0,1,1,1,0,0,1,0,1
4,1,0,0,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,1,1,0,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,...,0,1,1,0,0,0,1,0,1,1,0,1,1,1,0,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
266,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
267,1,1,0,1,1,0,0,1,1,1,0,1,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,1,1,1,0,1,1,1,0,0,1,1,1,...,1,1,0,0,0,1,1,1,0,0,0,1,1,1,1,0,1,0,0,1,1,1,0,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,1,1
268,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
pd.DataFrame(y_vali_)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
4,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
266,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
267,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
268,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dreem_sleep_apnea_custom_metric(pred, y_vali_)

0.029192124915139173